In [ ]:
# ============================================================
# 0. LIBRAIRIES ET CONFIGURATION
# ============================================================

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from shapely.geometry import shape

pd.set_option("display.max_columns", None)
sns.set_style("whitegrid")


In [ ]:
# ============================================================# 1. DONNÉES EHCVM – CONDITIONS DE VIE DES MÉNAGES# ============================================================ehcvm_file = "./data/ehcvm_menage_mli2021_clean.csv"ehcvm = pd.read_csv(ehcvm_file, sep=";", dtype=str, engine="python")# Harmonisation des noms de colonnesehcvm.columns = (    ehcvm.columns    .str.lower()    .str.replace(" ", "_")    .str.replace("[^0-9a-z_]", "", regex=True))# Variables socio-économiques retenuesconditions_vie_cols = ['menage', 'elec_ac', 'ordure', 'toilet']# Conversion en numériquesfor col in conditions_vie_cols:    ehcvm[col] = pd.to_numeric(ehcvm[col], errors='coerce')# Agrégation au niveau de la grappeehcvm_agg = (    ehcvm    .groupby("grappe")[conditions_vie_cols]    .mean()    .reset_index())


In [ ]:
# ============================================================# 2. DONNÉES SENTINEL-2 – INDICES SPECTRAUX (EXPORT GEE)# ============================================================sentinel_file = "./data/Indices_Spectraux_Mali_2021.csv"# Lecture tolérante aux erreurs de formatdf_raw = pd.read_csv(    sentinel_file,    sep=';',    engine='python',    on_bad_lines='skip',    dtype=str)# Colonnes utilesindices_cols = ['NDBI', 'NDMI', 'NDVI', 'SAVI', 'UI', '.geo']df = df_raw[indices_cols].copy()


In [ ]:
# ============================================================
# 3. CONVERSION GÉOMÉTRIQUE
# ============================================================

def geojson_to_geometry(geojson_str):
    try:
        return shape(json.loads(geojson_str))
    except:
        return None

df['geometry'] = df['.geo'].apply(geojson_to_geometry)
df = df.drop(columns=['.geo'])

gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.set_crs(epsg=4326, inplace=True)


In [ ]:
# ============================================================
# 4. NETTOYAGE ET VALIDATION DES INDICES SENTINEL
# ============================================================

indices = ['NDBI', 'NDMI', 'NDVI', 'SAVI', 'UI']

# Conversion numérique
for col in indices:
    gdf[col] = pd.to_numeric(gdf[col], errors='coerce')

# Suppression des géométries invalides
gdf = gdf[gdf.geometry.notnull()]
gdf = gdf[gdf.is_valid]

print("Nombre d'entités valides :", len(gdf))


In [ ]:
# ============================================================
# 5. STATISTIQUES DESCRIPTIVES
# ============================================================

stats_sentinel = gdf[indices].describe().T
print("\nStatistiques Sentinel :")
print(stats_sentinel)


In [ ]:
# ============================================================
# 6. DISTRIBUTIONS ET CORRÉLATIONS INTERNES
# ============================================================

# Histogrammes
gdf[indices].hist(bins=20, figsize=(12, 8))
plt.suptitle("Distributions des indices Sentinel-2", fontsize=14)
plt.show()

# Corrélations internes
corr_sentinel = gdf[indices].corr(method='pearson')
print("\nCorrélations internes Sentinel :")
print(corr_sentinel)


In [ ]:
# ============================================================
# 7. AGRÉGATION ET EXPORT FINAL
# ============================================================

sentinel_means = gdf[indices].mean()
print("\nMoyennes Sentinel :")
print(sentinel_means)

# Export final
gdf_clean = gdf[indices + ['geometry']]
gdf_clean.to_file("Sentinel_Mali_2021_READY.geojson", driver="GeoJSON")
gdf_clean.drop(columns='geometry').to_csv("Sentinel_Mali_2021_READY.csv", index=False)

print("Exports terminés")
